In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,15 mins 32 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_andyg_2dqnen
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.923 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import numpy as np
import sys
import matplotlib.pyplot as plt 
np.set_printoptions(threshold=sys.maxsize)

In [3]:
data = pd.read_csv('sequence_execution_time.csv')

In [4]:
data = data.drop(data.index[1000:])

In [5]:
reads = data.drop(['sequence_identifier','quality'], axis =1)
reads

,read_sequence,execution_time
0,CTGGTCCTAACATCTGTTGACCATAGCCATATTTTTCAGAAAAGTC...,2295.125
1,TGGCTATGGGCTGCAATAATTCGTCTTTCTGAAAGGGCATTCAGCC...,674.000
2,ACTTTCTATCTCTATGGATTTAACCATTGTGGATATTTCATGTAAA...,757.209
3,ATCAAAAACATTCAAATGAAATCATCTTAAGTGCCAGAATTATAAA...,397.042
4,CAGGTTCCAATATTACAAATGCAAGGCCAGAAGGAGACTGCAAATG...,675.000
...,...,...
995,ACAACTAGTTAGAGTCTTACAAAACAATAAAAACTTTGGGGTTAAC...,597.959
996,GGAGTTTGTGAGGTTGAAATAAACTTTTCTTAACTATCCATCATGA...,635.792
997,AACTTCTTTTGTTAAATTTATTCCTAAGGATTTTGTGTTTTTCAAT...,391.500
998,GCTTCCTCCAGGATGCATGAAGAAGCTATCGGAACCTCCTCCGACT...,528.458


In [6]:
#Normalize data
reads['execution_time'] = np.log10(reads.execution_time)
reads

,read_sequence,execution_time
0,CTGGTCCTAACATCTGTTGACCATAGCCATATTTTTCAGAAAAGTC...,3.360806
1,TGGCTATGGGCTGCAATAATTCGTCTTTCTGAAAGGGCATTCAGCC...,2.828660
2,ACTTTCTATCTCTATGGATTTAACCATTGTGGATATTTCATGTAAA...,2.879216
3,ATCAAAAACATTCAAATGAAATCATCTTAAGTGCCAGAATTATAAA...,2.598836
4,CAGGTTCCAATATTACAAATGCAAGGCCAGAAGGAGACTGCAAATG...,2.829304
...,...,...
995,ACAACTAGTTAGAGTCTTACAAAACAATAAAAACTTTGGGGTTAAC...,2.776671
996,GGAGTTTGTGAGGTTGAAATAAACTTTTCTTAACTATCCATCATGA...,2.803315
997,AACTTCTTTTGTTAAATTTATTCCTAAGGATTTTGTGTTTTTCAAT...,2.592732
998,GCTTCCTCCAGGATGCATGAAGAAGCTATCGGAACCTCCTCCGACT...,2.723010


In [7]:
def Kmers_funct(seq, size=9): 
   return [seq[x:x+size].upper() for x in range(len(seq) - size + 1)]

reads['words'] = reads.apply(lambda x: Kmers_funct(x['read_sequence']), axis=1)
reads = reads.drop('read_sequence', axis=1)
reads

,execution_time,words
0,3.360806,"[CTGGTCCTA, TGGTCCTAA, GGTCCTAAC, GTCCTAACA, T..."
1,2.828660,"[TGGCTATGG, GGCTATGGG, GCTATGGGC, CTATGGGCT, T..."
2,2.879216,"[ACTTTCTAT, CTTTCTATC, TTTCTATCT, TTCTATCTC, T..."
3,2.598836,"[ATCAAAAAC, TCAAAAACA, CAAAAACAT, AAAAACATT, A..."
4,2.829304,"[CAGGTTCCA, AGGTTCCAA, GGTTCCAAT, GTTCCAATA, T..."
...,...,...
995,2.776671,"[ACAACTAGT, CAACTAGTT, AACTAGTTA, ACTAGTTAG, C..."
996,2.803315,"[GGAGTTTGT, GAGTTTGTG, AGTTTGTGA, GTTTGTGAG, T..."
997,2.592732,"[AACTTCTTT, ACTTCTTTT, CTTCTTTTG, TTCTTTTGT, T..."
998,2.723010,"[GCTTCCTCC, CTTCCTCCA, TTCCTCCAG, TCCTCCAGG, C..."


In [8]:
human_texts = list(reads['words'])
for item in range(len(human_texts)):
   human_texts[item] = ' '.join(human_texts[item])
#separate labels
y_human = reads.iloc[:, 0].values # y_human for human_dna

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range = (4,4)) # initially (4,4)
X = cv.fit_transform(human_texts)

In [10]:
Y= y_human

X= X

In [ ]:
data

In [18]:
data = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
train, test = data.split_frame(ratios=[0.8], seed = 42)

In [20]:
####################################################################################
#x = train.columns
#y = 'revenue'
#x.remove(y)

In [21]:
model = H2OAutoML(sort_metric='rmse',
                  max_runtime_secs=120,
                  seed = 42,
                  max_models = 10,
               #   max_runtime_secs_per_model = 720,
                #  exclude_algos = ['DeepLearning'],
                  #stopping_metric = 'RMSLE'
                 )

In [22]:
model.train(training_frame = train, y = 'execution_time')

AutoML progress: |
01:00:54.130: AutoML: XGBoost is not available; skipping it.
01:00:54.141: _train param, Dropping bad and constant columns: [sequence_identifier, read_sequence, quality]
01:00:54.141: _train param, Training data must have at least 2 features (incl. response).
01:00:54.143: _train param, Dropping bad and constant columns: [sequence_identifier, read_sequence, quality]
01:00:54.144: _train param, Training data must have at least 2 features (incl. response).
01:00:54.148: _train param, Dropping bad and constant columns: [sequence_identifier, read_sequence, quality]
01:00:54.148: _train param, Training data must have at least 2 features (incl. response).
01:00:54.151: _train param, Dropping bad and constant columns: [sequence_identifier, read_sequence, quality]
01:00:54.151: _train param, Training data must have at least 2 features (incl. response).
01:00:54.154: _train param, Dropping bad and constant columns: [sequence_identifier, read_sequence, quality]
01:00:54.156: _

In [23]:
h2o_models = h2o.automl.get_leaderboard(model, extra_columns = "ALL")
h2o_models

H2OFrame is empty.

In [24]:
best_model = model.get_best_model()
print(best_model)

None


In [25]:
best_model.model_performance(test)

AttributeError: 'NoneType' object has no attribute 'model_performance'

In [ ]:
y_pred = model.predict(test)
y_pred

In [ ]:
y_truth = test['execution_time']
y_truth

In [ ]:
#y_pred.types

In [ ]:
explain_model = model.explain(frame = test, figsize = (8,6))

In [ ]:
# from h2o.model.regression import h2o_mean_squared_error

# h2o_mean_squared_error(y_truth, y_pred)